<img src="https://www.th-koeln.de/img/logo.svg" style="float:right;" width="200">

# 13th exercise: <font color="#C70039">First Reinforcement Learning Q-Table learning</font>
* Course: AML
* Lecturer: <a href="https://www.gernotheisenberg.de/">Gernot Heisenberg</a>
* Author of notebook: <a href="https://www.gernotheisenberg.de/">Gernot Heisenberg</a>
* Editor of notebook: Lena Pickartz
* Date: 17.01.2025

---------------------------------
**GENERAL NOTE 1**:
Please make sure you are reading the entire notebook, since it contains a lot of information on your tasks (e.g. regarding the set of certain paramaters or a specific computational trick), and the written mark downs as well as comments contain a lot of information on how things work together as a whole.

**GENERAL NOTE 2**:
* Please, when commenting source code, just use English language only.
* When describing an observation please use English language, too.
* This applies to all exercises throughout this course.

---------------------------------

### <font color="FFC300">TASKS</font>:
The tasks that you need to work on within this notebook are always indicated below as bullet points.
If a task is more challenging and consists of several steps, this is indicated as well.
Make sure you have worked down the task list and commented your doings.
This should be done by using markdown.<br>
<font color=red>Make sure you don't forget to specify your name and your matriculation number in the notebook.</font>

**YOUR TASKS in this exercise are as follows**:
1. import the notebook to Google Colab or use your local machine.
2. make sure you specified you name and your matriculation number in the header below my name and date.
    * set the date too and remove mine.
3. read the entire notebook carefully
    * add comments whereever you feel it necessary for better understanding
    * run the notebook for the first time.
4. play with all hyperparameters including the actions, states, rewards table.
5. add and implement an ϵ-greedy strategy
---------------------------------

In [1]:
# Imports
import numpy as np

### Create the possible states

In [2]:
location_to_state = {
    'L1' : 0,
    'L2' : 1,
    'L3' : 2,
    'L4' : 3,
    'L5' : 4,
    'L6' : 5,
    'L7' : 6,
    'L8' : 7,
    'L9' : 8
}

state_to_location = dict((state,location) for location, state in location_to_state.items())

### Create the actions & rewards

In [3]:
actions = [0,1,2,3,4,5,6,7,8]

In [4]:
rewards = np.array([[0,1,0,0,0,0,0,0,0],
                   [1,0,1,0,1,0,0,0,0],
                   [0,1,0,0,0,1,0,0,0],
                   [0,0,0,0,0,0,1,0,0],
                   [0,1,0,0,0,0,0,1,0],
                   [0,0,1,0,0,0,0,0,0],
                   [0,0,0,1,0,0,0,1,0],
                   [0,0,0,0,1,0,1,0,1],
                   [0,0,0,0,0,0,0,1,0]])

### Def remaining hyperparameters

In [5]:
# initialize the parameters
gamma = 0.99 # discount factor
alpha = 0.7  # learning rate

### Define agent and its attributes

In [6]:
class QAgent():
    # initialize everything
    def __init__(self, alpha, gamma, location_to_state, actions, rewards, state_to_location):
        self.gamma = gamma
        self.alpha = alpha

        self.location_to_state = location_to_state
        self.actions = actions
        self.rewards = rewards
        self.state_to_location = state_to_location

        # remember, the Q-value table is of size all actions x all states
        M = len(location_to_state)
        self.Q = np.zeros((M,M), dtype = None, order = 'C')

    # now, implement the training method for the agent
    def training(self, start_location, end_location, iterations):

        rewards_new = np.copy(self.rewards)

        ending_state = self.location_to_state[end_location]

        rewards_new[ending_state, ending_state] = 999

        # DEBUG
        print(rewards_new)

        # pick random current state
        # iterations = the # of training cycles
        for i in range(iterations):
            current_state = np.random.randint(0,9)
            playable_actions = []

            # iterate thru the rewards matrix to get states
            # that are really reachable from the randomly chosen
            # state and assign only those in a list since they are really playable
            for j in range(9):
                if rewards_new[current_state, j] > 0:
                    playable_actions.append(j)

            # choosing next random state
            # however, make sure that playable_actions is not empty
            if len(playable_actions) != 0:
                # Randomly select a value from playable_actions
                next_state = np.random.choice(playable_actions)

            # finding the difference in Q, often referred to as temporal difference
            # by means of the Bellman's equation (compare with slides)
            TD = rewards_new[current_state, next_state] + self.gamma * self.Q[next_state, np.argmax(self.Q[next_state,])] - self.Q[current_state, next_state]
            self.Q[current_state, next_state] += self.alpha*TD
            # DEBUG
            #print(f"Q[{current_state}, {next_state}]:", self.Q[current_state, next_state])

        route = [start_location]
        next_location = start_location

        # print the optimal route from start to end
        self.get_optimal_route(start_location, end_location, next_location, route, self.Q)

    # compute the optimal route
    def get_optimal_route(self, start_location, end_location, next_location, route, Q):
        while(next_location != end_location):
            starting_state = self.location_to_state[start_location]
            next_state = np.argmax(Q[starting_state,])
            next_location = self.state_to_location[next_state]
            route.append(next_location)
            start_location = next_location
        # DEBUG
        print('Q-table:',Q)
        print("optimal route:", route)

In [7]:
qagent = QAgent(alpha, gamma, location_to_state, actions, rewards, state_to_location)
qagent.training('L9', 'L4', 1000)

[[  0   1   0   0   0   0   0   0   0]
 [  1   0   1   0   1   0   0   0   0]
 [  0   1   0   0   0   1   0   0   0]
 [  0   0   0 999   0   0   1   0   0]
 [  0   1   0   0   0   0   0   1   0]
 [  0   0   1   0   0   0   0   0   0]
 [  0   0   0   1   0   0   0   1   0]
 [  0   0   0   0   1   0   1   0   1]
 [  0   0   0   0   0   0   0   1   0]]
Q-table: [[    0.         24529.75520626     0.             0.
      0.             0.             0.             0.
      0.        ]
 [23062.73080131     0.         23085.68906258     0.
  25299.20493687     0.             0.             0.
      0.        ]
 [    0.         23328.48652325     0.             0.
      0.         22791.6140084      0.             0.
      0.        ]
 [    0.             0.             0.         31054.48413566
      0.             0.         28572.31004584     0.
      0.        ]
 [    0.         24531.63211626     0.             0.
      0.             0.             0.         26305.21559313
      0.   

# Play Around with Hyperparameters

In [8]:
# initialize the parameters
gamma = 0.01 # discount factor
alpha = 0.99  # learning rate

In [9]:
qagent = QAgent(alpha, gamma, location_to_state, actions, rewards, state_to_location)
qagent.training('L9', 'L4', 1000)

[[  0   1   0   0   0   0   0   0   0]
 [  1   0   1   0   1   0   0   0   0]
 [  0   1   0   0   0   1   0   0   0]
 [  0   0   0 999   0   0   1   0   0]
 [  0   1   0   0   0   0   0   1   0]
 [  0   0   1   0   0   0   0   0   0]
 [  0   0   0   1   0   0   0   1   0]
 [  0   0   0   0   1   0   1   0   1]
 [  0   0   0   0   0   0   0   1   0]]
Q-table: [[   0.            1.01010111    0.            0.            0.
     0.            0.            0.            0.        ]
 [   1.01010101    0.            1.01010101    0.            1.01011109
     0.            0.            0.            0.        ]
 [   0.            1.01010111    0.            0.            0.
     1.01010101    0.            0.            0.        ]
 [   0.            0.            0.         1009.09090909    0.
     0.            1.11090909    0.            0.        ]
 [   0.            1.01010111    0.            0.            0.
     0.            0.            1.01110909    0.        ]
 [   0.         

In [10]:
# initialize the parameters
gamma = 0.99 # discount factor
alpha = 0.01  # learning rate

In [11]:
qagent = QAgent(alpha, gamma, location_to_state, actions, rewards, state_to_location)
qagent.training('L9', 'L4', 1000)

[[  0   1   0   0   0   0   0   0   0]
 [  1   0   1   0   1   0   0   0   0]
 [  0   1   0   0   0   1   0   0   0]
 [  0   0   0 999   0   0   1   0   0]
 [  0   1   0   0   0   0   0   1   0]
 [  0   0   1   0   0   0   0   0   0]
 [  0   0   0   1   0   0   0   1   0]
 [  0   0   0   0   1   0   1   0   1]
 [  0   0   0   0   0   0   0   1   0]]
Q-table: [[0.00000000e+00 8.61980747e-01 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [5.16567549e-01 0.00000000e+00 4.31508732e-01 0.00000000e+00
  5.61988418e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [0.00000000e+00 5.26067790e-01 0.00000000e+00 0.00000000e+00
  0.00000000e+00 6.22283383e-01 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 5.08218353e+02
  0.00000000e+00 0.00000000e+00 1.85805343e+01 0.00000000e+00
  0.00000000e+00]
 [0.00000000e+00 5.18715155e-01 0.00000000e+00 0.00000000e+00
  0.00000

The high learning rate in Run 1 led to more uniform Q-values across non-goal states and has quicker convergence to similar values.

The high discount factor in Run 2 resulted in more varied Q-values, a better differentiation between paths and astronger value propagation from the goal state.

The reward structure (shown in the adjacency matrix) has a high reward of 999 at the goal state (L4), which both configurations successfully learned to reach.

# ϵ-greedy strategy

In [41]:
class QAgent():
    def __init__(self, alpha, gamma, epsilon, location_to_state, actions, rewards, state_to_location):
        self.gamma = gamma
        self.alpha = alpha
        self.epsilon = epsilon
        self.location_to_state = location_to_state
        self.actions = actions
        self.rewards = rewards
        self.state_to_location = state_to_location
        self.states = list(self.location_to_state.values())
        self.Q = np.zeros((len(self.states), len(self.states)))

    def get_available_actions(self, state):
        """Helper function to get valid actions for a state"""
        return [i for i in range(len(self.states)) if self.rewards[state, i] > 0]

    def choose_action(self, state, available_actions):
        """Choose action using ε-greedy strategy"""
        if not available_actions:
            print(f"Warning: No available actions from state {state}")
            return None

        if np.random.random() < self.epsilon:
            return np.random.choice(available_actions)
        else:
            state_q_values = self.Q[state][available_actions]
            return available_actions[np.argmax(state_q_values)]

    def training(self, start_location, end_location, iterations):
        rewards_new = np.copy(self.rewards)
        ending_state = self.location_to_state[end_location]
        start_state = self.location_to_state[start_location]

        # Set high reward for reaching goal
        for i in range(len(self.states)):
            if rewards_new[i, ending_state] > 0:
                rewards_new[i, ending_state] = 999

        for episode in range(iterations):
            if episode % 100 == 0:
                print(f"Episode {episode}/{iterations}")

            current_state = start_state
            steps = 0
            path = [current_state]

            while current_state != ending_state and steps < 20:  # Limit steps per episode
                available_actions = self.get_available_actions(current_state)

                if not available_actions:
                    print(f"Dead end at state {current_state}")
                    break

                action = self.choose_action(current_state, available_actions)
                if action is None:
                    break

                # Get reward and update Q-value
                reward = rewards_new[current_state, action]
                next_available_actions = self.get_available_actions(action)

                if next_available_actions:
                    next_max_q = np.max(self.Q[action][next_available_actions])
                else:
                    next_max_q = 0

                # Q-learning update
                self.Q[current_state, action] = (1 - self.alpha) * self.Q[current_state, action] + \
                                              self.alpha * (reward + self.gamma * next_max_q)

                current_state = action
                path.append(current_state)
                steps += 1

        # Get optimal route
        route = self.get_optimal_route(start_location, end_location)
        print("Q-table:")
        print(self.Q)
        print("Final route:", route)
        return route

    def get_optimal_route(self, start_location, end_location):
        route = [start_location]
        current_location = start_location

        while current_location != end_location:
            current_state = self.location_to_state[current_location]
            available_actions = self.get_available_actions(current_state)

            if not available_actions:
                print(f"No valid actions from {current_location}")
                break

            next_state = available_actions[np.argmax(self.Q[current_state][available_actions])]
            current_location = self.state_to_location[next_state]
            route.append(current_location)

            if len(route) > 20:  # Prevent infinite loops
                print("Route finding exceeded maximum length")
                break

        return route

In [42]:
# initialize the parameters
gamma = 0.99  # discount factor
alpha = 0.7  # learning rate
epsilon = 0.1  # exploration rate (10% exploration)

In [43]:
# Create agent with epsilon parameter added
qagent = QAgent(alpha, gamma, epsilon, location_to_state, actions, rewards, state_to_location)
qagent.training('L9', 'L4', 1000)

Episode 0/1000
Episode 100/1000
Episode 200/1000
Episode 300/1000
Episode 400/1000
Episode 500/1000
Episode 600/1000
Episode 700/1000
Episode 800/1000
Episode 900/1000
Q-table:
[[  0.          96.51164373   0.           0.           0.
    0.           0.           0.           0.        ]
 [ 96.54169024   0.          92.0383249    0.         957.95082072
    0.           0.           0.           0.        ]
 [  0.          94.1851596    0.           0.           0.
    0.           0.           0.           0.        ]
 [  0.           0.           0.           0.           0.
    0.           0.           0.           0.        ]
 [  0.         855.00116443   0.           0.           0.
    0.           0.         981.10988665   0.        ]
 [  0.           0.           0.           0.           0.
    0.           0.           0.           0.        ]
 [  0.           0.           0.         999.           0.
    0.           0.         981.1099       0.        ]
 [  0.           

['L9', 'L8', 'L7', 'L4']

# Oberservations

- Multiple paths were explored (shown by various non-zero Q-values)
- Clear convergence to optimal path (shown by very high values ~957-999)
- Some sub-optimal paths were explored but got lower values
- Found optimal route: L9 → L8 → L7 → L4
- High Q-values (~980-999) along this path show strong confidence
- 10% exploration rate was enough to find and verify this was the best path